# Préambule

Paul et Béatrice ont préparé les données des exports Toflit18 pour les provinces + dans carte matricielle voir quel port exporte quoi et à quel niveau (+ même chose en import).

Objectif : transposer ces données en carte au niveau province.

- par province, avoir le camembert des imports et exports
- faire sauter aux yeux les provinces exportatrices et les provinces exportatrices

In [ ]:
# Load an empty map
from keplergl import KeplerGl
from poitousprint import Portic, get_online_csv
import json
import pandas as pd
import geopandas

In [ ]:
# step 1 : récupérer les données

mappings_list = get_online_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQcJoe-LTCNLyIAKIQoOTevylrZG2Adn2sTbNtuXjn5wQRQLMlVfIbQHPoyqu3mc6-Um4jjnBPV46VI/pub?output=csv")

mappings = {}
for m in mappings_list:
    # mappings[m['direction_fermes']] = [*mappings[m['direction_fermes']], m['province']] if m['direction_fermes'] in mappings else [m['province']]
    if m['direction_fermes'] not in mappings :
        mappings[m['direction_fermes']] = m['province']
mappings

In [ ]:
# step 2 : préparation pour la visualisation
imports = pd.read_csv("imports.csv")
exports = pd.read_csv("exports.csv")


def clean_and_join (df):
    df["source"] = df.source.str[3:]
    df["target"] = df.target.str[3:]
    df['source_province'] = df.source.map(mappings)
    return df

def sum_by_province (df) :
    return df.groupby('source_province')['value'].sum()
    
imports = clean_and_join(imports)
exports = clean_and_join(exports)

imports.source_province.value_counts()

exports_summed = sum_by_province(exports)
imports_summed = sum_by_province(imports)

exports_summed.to_csv('exports_summed.csv')
imports_summed.to_csv('imports_summed.csv')



In [ ]:
# step 3 : charger le fond de carte et faire la jointure avec les données d'imports et d'exports

polygons = geopandas.read_file("cartoweb_france_1789_geojson.geojson")
polygons["import_values"] = polygons.shortname.map(imports_summed).fillna(0)
polygons["export_values"] = polygons.shortname.map(exports_summed).fillna(0)

In [ ]:
# step 3 : charger le fond de carte et faire la jointure avec les données d'imports et d'exports

polygons = geopandas.read_file("cartoweb_france_1789_geojson.geojson")
polygons["import_values"] = polygons.shortname.map(imports_summed).fillna(0)
polygons["export_values"] = polygons.shortname.map(exports_summed).fillna(0)

# step 4 : visualisation

config = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

"""
# create data frame
df_with_geometry = pd.DataFrame({
    'id': [1],
    'geometry_string': [json.dumps(background)]
})

# add to map
map_1.add_data(df_with_geometry, "df_with_geometry")
"""


map = KeplerGl(config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "vf95r17",
          "type": "geojson",
          "config": {
            "dataId": "feature_collection",
            "label": "feature_collection",
            "color": [
              34,
              63,
              154
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": [
                218,
                112,
                191
              ],
              "colorRange": {
                "name": "Custom Palette",
                "type": "custom",
                "category": "Custom",
                "colors": [
                  "#FFFFF",
                  "#FFFFF",
                  "#9DD0D4",
                  "#75BBC1",
                  "#4BA7AF",
                  "#00939C",
                  "#108188"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": True,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "import_values",
              "type": "real"
            },
            "colorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "feature_collection": [
              {
                "name": "id",
                "format": None
              },
              {
                "name": "unitlevel",
                "format": None
              },
              {
                "name": "shortname",
                "format": None
              },
              {
                "name": "id_sup",
                "format": None
              },
              {
                "name": "import_values",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 47.47515841923549,
      "longitude": -5.2770757698695085,
      "pitch": 0,
      "zoom": 3.874468584124823,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}, data={"feature_collection": polygons} )
# map.add_data(data=polygons, name="feature_collection")

map

In [ ]:
{
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "ztctw0m",
          "type": "geojson",
          "config": {
            "dataId": "feature_collection",
            "label": "feature_collection",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": [
                221,
                178,
                124
              ],
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": True,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "import_values",
              "type": "real"
            },
            "colorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "feature_collection": [
              {
                "name": "id",
                "format": None
              },
              {
                "name": "unitlevel",
                "format": None
              },
              {
                "name": "shortname",
                "format": None
              },
              {
                "name": "id_sup",
                "format": None
              },
              {
                "name": "import_values",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 45.6876849,
      "longitude": -1.15,
      "pitch": 0,
      "zoom": 5.2,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}